In [1]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect("companyDb.db")
cursor = connection.cursor()

# Enable foreign key constraints in SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

# Create Department table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Department (
    Dname TEXT NOT NULL,
    Dnumber INTEGER PRIMARY KEY,
    Mang_ssn INTEGER,
    mgr_start DATE,
    FOREIGN KEY (Mang_ssn) REFERENCES Employee(SSN) ON DELETE SET NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Employee (
    SSN INTEGER PRIMARY KEY,
    Fname TEXT NOT NULL,
    Lname TEXT NOT NULL,
    gender TEXT CHECK(gender IN ('Male', 'Female')),
    address TEXT,
    salary REAL,
    Super_SSN INTEGER,
    Dno INTEGER,
    FOREIGN KEY (Super_SSN) REFERENCES Employee(SSN) ON DELETE SET NULL,
    FOREIGN KEY (Dno) REFERENCES Department(Dnumber) ON DELETE SET NULL
);
""")

# Insert data into Department table
cursor.executemany("""
INSERT INTO Department (Dname, Dnumber, Mang_ssn, mgr_start) VALUES (?, ?, ?, ?);
""", [
    ("HR", 1, None, "2022-01-15"),
    ("IT", 2, None, "2023-05-20"),
    ("Sales", 3, None, "2021-09-10"),
])

# Insert data into Employee table
cursor.executemany("""
INSERT INTO Employee (SSN, Fname, Lname, gender, address, salary, Super_SSN, Dno) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
""", [
    (101, "John", "Doe", "Male", "123 Elm St", 3000, None, 1),
    (102, "Jane", "Smith", "Female", "456 Oak St", 4500, None, 2),
    (103, "Alice", "Johnson", "Female", "789 Pine St", 5000, None, 3),
    (104, "Bob", "Brown", "Male", "101 Maple St", 2500, None, 1),
    (105, "Eve", "Davis", "Female", "202 Birch St", 6000, None, 2),
])

# Commit the changes
connection.commit()

Question: 
Write a SQL query to retrieve the emp_id, last_name, and salary of employees whose salary is 
between 2,000 and 5,000 and do not have a manager ID of 101 or 200. 
Instructions: 
1. Use the SELECT statement to specify the columns: emp_id, last_name, and salary. 
2. Filter the results using the WHERE clause with the BETWEEN operator to set the salary range. 
3. Use the NOT IN clause to exclude certain manager IDs. 
4. Combine conditions using the AND logical operator. 

In [2]:
cursor.execute("SELECT Fname, Lname, salary FROM Employee WHERE (salary BETWEEN 2000 AND 5000) or (Super_SSN != 101 AND Super_SSN != 200)")
print(cursor.fetchall())

[('John', 'Doe', 3000.0), ('Jane', 'Smith', 4500.0), ('Alice', 'Johnson', 5000.0), ('Bob', 'Brown', 2500.0)]


Question: 
Write a SQL query to display the employee names along with their respective department names. 
Use aliases for table names for better readability. 
Instructions: 
1. Use the SELECT statement to specify the columns: employee.name and department.name. 
2. Use the FROM clause to include the tables employees and departments. 
3. Use an INNER JOIN to connect the employees and departments tables based on the 
department IDs. 
4. Use table aliases (e.g., e for employees, d for departments) to shorten the table names in the 
query. 
5. Order the results by department name in ascending order. 

In [ ]:
cursor.execute("""
SELECT e.Fname , e.Lname AS EmployeeName, d.Dname AS DepartmentName
FROM Employee AS e
INNER JOIN Department AS d
ON e.Dno = d.Dnumber
ORDER BY d.Dname ASC;
""")

print (cursor.fetchall())

Write a SQL query to find the number of employees and the average salary for each department. 
Ensure that the results are grouped by department ID. 
Instructions: 
1. Use the SELECT statement to specify the department ID, the count of employees, and the 
average salary. 
2. Use the GROUP BY clause to group the results by department ID. 
3. Use the COUNT() function to find the number of employees in each department. 
4. Use the AVG() function to calculate the average salary in each department.

In [5]:
cursor.execute("select d.Dnumber , Count(e.SSN) as 'countofemployees' , avg(e.salary) as 'avgsalary'  from Department d left join Employee e on e.Dno = d.Dnumber group by d.Dnumber  ")

print(cursor.fetchall())

[(1, 2, 2750.0), (2, 2, 5250.0), (3, 1, 5000.0)]
